In [1]:
import pandas as pd
import datetime as dt

Downloaded data from: http://web.mta.info/developers/turnstile.html

Documentations at: http://web.mta.info/developers/resources/nyct/turnstile/ts_Field_Description.txt

Map of the MTA system: http://web.mta.info/maps/submap.html

In [2]:
df1 = pd.read_csv('turnstile_180526.txt')
df2 = pd.read_csv('turnstile_180602.txt')
df3 = pd.read_csv('turnstile_180609.txt')
df4 = pd.read_csv('turnstile_180616.txt')

In [3]:
df = pd.concat([df1, df2, df3, df4])

In [4]:
df.reset_index(inplace=True, drop=True)

Fix 'EXITS' column name (it has many spaces at the end of it)

In [5]:
df.columns = df.columns.str.strip()

In [6]:
df.to_csv('raw_turnstile_data.csv')

In [7]:
df['DATETIME'] = pd.to_datetime(df.DATE + ' ' + df.TIME, format='%m/%d/%Y %H:%M:%S')
df['DATE'] = pd.to_datetime(df['DATE'], format='%m/%d/%Y')

In [8]:
df['STATION_KEY'] = df['C/A'] + ' ' + df['UNIT'] + ' ' + df['STATION']

In [ ]:
# cols_to_drop = ['C/A','UNIT','STATION']
# cols_to_drop = ['C/A','UNIT']
# df.drop(cols_to_drop, axis=1, inplace=True)

Sort, Group by Station Key, then compute difference between 'ENTRIES'

In [9]:
df.sort_values(['STATION_KEY', 'SCP', 'DATETIME'], inplace=True)

In [10]:
df['ENTRY_DIFFS'] = df.groupby(['STATION_KEY','SCP'])['ENTRIES'].diff(periods=-1)*-1

In [11]:
df['EXIT_DIFFS'] = df.groupby(['STATION_KEY','SCP'])['EXITS'].diff(periods=-1)*-1

The majority of 'ENTRY_DIFFS' values are reasonable, but there are still some large negative and positive values still. For now, exclude them from our analysis. (There shouldn't, in fact, be any negative values.)

In [21]:
# 200,000 entries per 4-hr time period is a reasonable cut-off
clean_df = df[(df['ENTRY_DIFFS'] < 2E5) 
              & (df['ENTRY_DIFFS'] > 0) 
              & (df['EXIT_DIFFS'] < 2E5)
              & (df['EXIT_DIFFS'] > 0)]

In [30]:
clean_df.to_csv('clean_turnstile_data.csv')